# KubeFlow Pipeline: Github Issue Summarization using Tensor2Tensor

Currently, this notebook must be run from the Kubeflow JupyterHub installation, as described in the codelab.

In this notebook, we will show how to:

* Interactively define a KubeFlow Pipeline using the Pipelines Python SDK
* Submit and run the pipeline
* Add a step in the pipeline

This example pipeline trains a [Tensor2Tensor](https://github.com/tensorflow/tensor2tensor/) model on Github issue data, learning to predict issue titles from issue bodies. It then exports the trained model and deploys the exported model to [Tensorflow Serving](https://github.com/tensorflow/serving). 
The final step in the pipeline launches a web app which interacts with the TF-Serving instance in order to get model predictions.

## Enviroinment Setup

Before any experiment can be conducted. We need to setup and initialize an environment: ensure all Python modules has been setup and configured, as well as python modules

Setting up python modules

In [ ]:
!pip3 install --upgrade 'https://storage.googleapis.com/ml-pipeline/release/0.1.9/kfp.tar.gz' > /dev/null
!pip3 install --upgrade './extensions' > /dev/null
%load_ext extensions

All imports goes here

In [ ]:
import boto3
import kfp
from kfp import compiler
import kfp.dsl as dsl
import kfp.gcp as gcp
import kfp.notebook

from ipython_secrets import get_secret
from kfp.compiler import Compiler

import extensions
import extensions.kaniko as kaniko

Do some imports and set some variables.  Set the `WORKING_DIR` to a path under the Cloud Storage bucket you created earlier.

In [ ]:
EXPERIMENT_NAME = 'Github issue summarization'

USER = environ['JUPYTERHUB_USER']
GITHUB_TOKEN = get_secret('GITHUB_TOKEN')
# AWS_S3_BUCKET = get_secret('AWS_S3_BUCKET')
AWS_S3_BUCKET = 'bucket.cluster2.antoncloud1.superhub.io'

DEPLOY_WEBAPP = 'false'
DOCKER_REGISTRY = get_secret('DOCKER_REGISTRY')
DOCKER_REGISTRY_SECRET = get_secret('DOCKER_REGISTRY_SECRET')
# we need to rotate image versions unless KFP dsl will support pullSecret
DOCKER_TAG = 'v12'

# we reuse docker tag as a safepoint for generated data
WORKING_DIR = f"s3://{AWS_S3_BUCKET}/{USER}/{DOCKER_TAG}"

# Uncomment this if you want to use 2Mi data set instead of 3Gi
DATA_SET = 'https://s3.us-east-2.amazonaws.com/asi-kubeflow-models/gh-issues/data-sample.csv'
# DATA_SET = 'https://s3.us-east-2.amazonaws.com/asi-kubeflow-models/gh-issues/data-full.csv'

GPU_SUPPORT = False
if GPU_SUPPORT:
    TENSORFLOW_IMAGE='tensorflow/tensorflow:latest-gpu'
else:
    TENSORFLOW_IMAGE='tensorflow/tensorflow:latest'

The Kubeflow Pipeline (KFP) system requires an "Experiment" to group pipeline runs. 

To get reference to experiment we try naive but idempotent method. If experiment with desired name does not exists then retrieval function will `get_experiment()` with raise `ValueError`. In this case we will create a new KFP experiment. 

In [ ]:
client = kfp.Client()
try:
    exp = client.get_experiment(experiment_name=EXPERIMENT_NAME)
except:
    exp = client.create_experiment(EXPERIMENT_NAME)

## Prepare images

Before we can run training, we will build and compile docker container that we will use later in our pipeline

### Prepare dockerfile templates

Docker images can be rendered via `%%template` or `%templatefile` magics. It can intelligently use mustache `{{placeholder}}` templating syntax. Content will be replaced by the user namespace defined variable or system environment variable

You can use flags with the magic function:
* `-v` - to see content of rendered file. 
* `-h` - for more options


In [ ]:
%%template Dockerfile.tf
FROM ubuntu:16.04
ENV PATH $PATH:/tools/ks/bin
RUN apt-get update -y
RUN apt-get install --no-install-recommends -y -q ca-certificates python-dev python-setuptools wget unzip
RUN easy_install pip
RUN pip install pyyaml==3.12 six==1.11.0 requests==2.18.4 tensorflow==1.11.0
RUN pip install boto3
RUN wget -nv https://github.com/ksonnet/ksonnet/releases/download/v0.11.0/ks_0.11.0_linux_amd64.tar.gz && \
    tar -xvzf ks_0.11.0_linux_amd64.tar.gz && \
    mkdir -p /tools/ks/bin && \
    cp ./ks_0.11.0_linux_amd64/ks /tools/ks/bin && \
    rm ks_0.11.0_linux_amd64.tar.gz && \
    rm -r ks_0.11.0_linux_amd64

In [ ]:
%%template Dockerfile.t2t
FROM tensorflow/tensorflow:latest
ENV PATH $PATH:/tools/node/bin
RUN apt-get update -y
RUN apt-get install --no-install-recommends -y -q ca-certificates python-dev python-setuptools \
                                                  wget unzip git
RUN easy_install pip
RUN pip install boto3
RUN pip install tensor2tensor
RUN pip install pyyaml==3.12 six==1.11.0

In [ ]:
%%template Dockerfile.serve
FROM {{DOCKER_REGISTRY}}/library/tf:{{DOCKER_TAG}}
COPY serving /ml
ENTRYPOINT ["python", "/ml/deploy-tf-serve.py"]

In [ ]:
%%template Dockerfile.deploy
FROM {{DOCKER_REGISTRY}}/library/tf:{{DOCKER_TAG}}
COPY webapp /ml
ENTRYPOINT ["python", "/ml/deploy-webapp.py"]

In [ ]:
%%template Dockerfile.dataprep
FROM {{DOCKER_REGISTRY}}/library/t2t:{{DOCKER_TAG}}
COPY preproc /ml
RUN mkdir -p /ml/gh_data
RUN mkdir -p /ml/gh_data/tmp
WORKDIR /ml
ENTRYPOINT ["python", "/ml/datagen.py"]

In [ ]:
%%template Dockerfile.webapp
FROM {{DOCKER_REGISTRY}}/library/t2t:{{DOCKER_TAG}}
# RUN pip install tensorflow_hub
RUN pip install tensorflow-serving-api
RUN pip install gunicorn
RUN pip install pandas
COPY webapp /ml
WORKDIR /ml/app
CMD gunicorn -w 4 -b :8080 main:app

In [ ]:
%%template Dockerfile.train
FROM {{TENSORFLOW_IMAGE}}
RUN apt-get update -y
RUN apt-get install --no-install-recommends -y -q ca-certificates python-dev python-setuptools wget unzip git
RUN easy_install pip
RUN pip install tensor2tensor
RUN pip install tensorflow_hub
RUN pip install pyyaml==3.12 six==1.11.0
COPY training /ml
ENTRYPOINT ["python", "/ml/train_model.py"]

### Define build pipeline

Define build pipeline. Yes, we arguably using KFP to build images  that will be de-facto used by final pipeline.

We use [Kaniko](https://github.com/GoogleContainerTools/kaniko) and Kubernetes to handle build operations. Build status can be tracked via KFP pipeline dashboard

In fact build image job can be even combined with primary pipeline as physically it will be different Kubernetes pods. However for sake of general purpose efficiency we schedule build process via separate pipeline step

In [ ]:
sess = boto3.session.Session()
build_ctx=f"s3://{AWS_S3_BUCKET}/{EXPERIMENT_NAME}/dockerbuild.tar.gz"

def kaniko_op(name, destination, dockerfile,
              context=build_ctx, aws_secret='jupyter-awscreds', 
              pull_secret=DOCKER_REGISTRY_SECRET):
    """ template function for kaniko build operation
    """
    return dsl.ContainerOp(
        name=name,
        image='gcr.io/kaniko-project/executor:latest',
        arguments=['--destination', destination,
                   '--dockerfile', dockerfile,
                   '--context', context]
    ).apply(
        kaniko.use_aws_credentials(secret_name=aws_secret)
    ).apply(
        kaniko.use_pull_secret(secret_name=pull_secret)
    )
    

@dsl.pipeline(
  name='Pipeline images',
  description='Build images that will be used by the pipeline'
)
def build_images():
    kaniko.upload_build_context_to_s3(build_ctx)
    kaniko.aws_to_kube_secret(secret_name='jupyter-awscreds', session=sess)
    t2t = kaniko_op(
        name='tensor2tensor',
        destination=f"{DOCKER_REGISTRY}/library/t2t:{DOCKER_TAG}",
        dockerfile='Dockerfile.t2t'
    )
    tf = kaniko_op(
        name='tensorflow-cpu',
        destination=f"{DOCKER_REGISTRY}/library/tf:{DOCKER_TAG}",
        dockerfile='Dockerfile.tf'
    )
    deploy = kaniko_op(
        name='deploy',
        destination=f"{DOCKER_REGISTRY}/library/deploy:{DOCKER_TAG}",
        dockerfile='Dockerfile.deploy'
    )

    serve = kaniko_op(
        name='serve',
        destination=f"{DOCKER_REGISTRY}/library/serve:{DOCKER_TAG}",
        dockerfile='Dockerfile.serve'
    )
    
    dataprep = kaniko_op(
        name='dataprep',
        destination=f"{DOCKER_REGISTRY}/library/dataprep:{DOCKER_TAG}",
        dockerfile='Dockerfile.dataprep'
    )

    webapp = kaniko_op(
        name='webapp',
        destination=f"{DOCKER_REGISTRY}/library/webapp:{DOCKER_TAG}",
        dockerfile='Dockerfile.webapp'
    )
        
    kaniko_op(
        name='train',
        destination=f"{DOCKER_REGISTRY}/library/train:{DOCKER_TAG}",
        dockerfile='Dockerfile.train',
    )
    # define dependencies
    deploy.after(tf)
    serve.after(tf)
    dataprep.after(t2t)
    webapp.after(t2t)
    
Compiler().compile(build_images, 'kaniko.tar.gz')

Compiler transforms Python DSL into an [Argo Workflow](https://argoproj.github.io/docs/argo/readme.html). And stores generated artifacts in `kaniko.tar.gz`. So it could be executed multiple times. Perhaps with different parameters

In [ ]:
run = client.run_pipeline(exp.id, 'Build images', 'kaniko.tar.gz')

Build process can be long a long term. Because often images that has been used for data science tasks are huge. In this case you might want to adjust `timeout` parameter

In [ ]:
# block till completion
client.wait_for_run_completion(run.id, timeout=720).run.status

# Data Preparation

Data preparation contains quite simple but yet powerful experiment. We will download CSV file from remote location and process data so, it can be consumed by our main pipeline. If needed both pipelines can be combined together. It makes sense to split data preparation and training pipelines because data have it's own lifecycle which is different to the model training

## Define Pipeline


In [ ]:
@dsl.pipeline(
  name='Data preparation',
  description="""Extract validate transform and load data into object storage. 
  So it could be accessible by the actual training
  """
)
def prepare_data(data_set: dsl.PipelineParam, data_gen: dsl.PipelineParam):
    dsl.ContainerOp(
        name='dataprep',
        image=f"{DOCKER_REGISTRY}/library/dataprep:{DOCKER_TAG}",
        arguments=['--data-set', data_set,
                   '--data-gen', data_gen]
    ).apply(
        # we want to use the same aws secret 
        # because we want to reuse same bucket as we have used for kaniko
        kaniko.use_aws_credentials(secret_name='jupyter-awscreds')
    )
#     dsl.get_pipeline_conf().set_image_pull_secrets([DOCKER_REGISTRY_SECRET])

Compiler().compile(prepare_data, 'dataprep.tar.gz')

In [ ]:
run = client.run_pipeline(exp.id, 'Prepare data', 'dataprep.tar.gz',
                          params={'data-set': DATA_SET,
                                  'data-gen': f"{WORKING_DIR}/gh_data"},
)

In [ ]:
# block till completion
client.wait_for_run_completion(run.id, timeout=720).run.status

## Define a Pipeline

Authoring a pipeline is like authoring a normal Python function. The pipeline function describes the topology of the pipeline. 

Each step in the pipeline is typically a `ContainerOp` --- a simple class or function describing how to interact with a docker container image. In the pipeline, all the container images referenced in the pipeline are already built. 

The pipeline starts by training a [Tensor2Tensor](https://github.com/tensorflow/tensor2tensor/) model, using already-preprocessed data. (More accurately, this step starts from an existing model checkpoint, then trains for a few more hundred steps).  When it finishes, it exports the model in a form suitable for serving by [TensorFlow serving](https://github.com/tensorflow/serving/).

The next step deploys a TF-serving instance with that model.

The last step launches a web app with which you can interact with the TF-serving instance to get model predictions.

In [ ]:
@dsl.pipeline(
    name='Github issue summarization',
    description='Tensor2Tensor-based training and TF-Serving'
)
def gh_summ(
    train_steps: dsl.PipelineParam=dsl.PipelineParam(name='train-steps', value=2019300),
    github_token: dsl.PipelineParam=dsl.PipelineParam(name='github-token', value='YOUR_GITHUB_TOKEN_HERE'),
    working_dir: dsl.PipelineParam=dsl.PipelineParam(name='working-dir', value='S3_LOCATION_GOES_HERE'),
    checkpoint_dir: dsl.PipelineParam=dsl.PipelineParam(name='checkpoint-dir', value='s3://ml-demo10/ghs_data/model_output_tbase.bak2019000'),
    deploy_webapp: dsl.PipelineParam=dsl.PipelineParam(name='deploy-webapp', value='true'),
    data_dir: dsl.PipelineParam=dsl.PipelineParam(name='data-dir', value='s3://ml-demo10/ghs_data/t2t_data_gh_all/')):

    train = dsl.ContainerOp(
        name = 'train',
        image = f"{DOCKER_REGISTRY}/library/train:{DOCKER_TAG}",
        arguments = [ 
            "--data-dir", data_dir,
            "--checkpoint-dir", checkpoint_dir,
            "--model-dir", f"{working_dir}/{{workflow.name}}/model_output",
            "--train-steps", train_steps, 
            "--deploy-webapp" , deploy_webapp],
        file_outputs={'output': '/tmp/output'}
        ).apply(
            # we want to use the same aws secret 
            # because we want to reuse same bucket as we have used for kaniko
            kaniko.use_aws_credentials(secret_name='jupyter-awscreds')
        )

#     serve = dsl.ContainerOp(
#         name = 'serve',
#         image = 'gcr.io/google-samples/ml-pipeline-kubeflow-tfserve',
#         arguments = ["--model_name", 'ghsumm-%s' % ('{{workflow.name}}',),
#           "--model_path", '%s/%s/model_output/export' % (working_dir, '{{workflow.name}}')
#           ]
#         )
#     with dsl.Condition(train.output=='true'):
#         webapp = dsl.ContainerOp(
#             name = 'webapp',
#             image = 'gcr.io/google-samples/ml-pipeline-webapp-launcher',
#             arguments = ["--model_name", 'ghsumm-%s' % ('{{workflow.name}}',),
#                 "--github_token", github_token]
#             )

#     serve.after(train)
#     webapp.after(serve)
#     if GPU_SUPPORT:
#         train.set_gpu_limit(4)


## Submit an experiment *run*

In [ ]:
compiler.Compiler().compile(gh_summ, 'ghsumm.tar.gz')

The call below will run the compiled pipeline.  We won't actually do that now, but instead we'll add a new step to the pipeline, then run it.

In [ ]:
# You'd uncomment this call to actually run the pipeline. 
run = client.run_pipeline(exp.id, 'ghsumm', 'ghsumm.tar.gz',
                          params={'working-dir': WORKING_DIR,
                                  'github-token': GITHUB_TOKEN})

In [ ]:
# block till completion
resp = client.wait_for_run_completion(run.id, timeout=720).run
resp.error or resp.status

## Add a step to the pipeline

Next, let's add a new step to the pipeline above.  As currently defined, the pipeline accesses a directory of pre-processed data as input to training.  Let's see how we could include the pre-processing as part of the pipeline. 

We're going to cheat a bit, as processing the full dataset will take too long for this workshop, so we'll use a smaller sample. For that reason, you won't actually make use of the generated data from this step (we'll stick to using the full dataset for training), but this shows how you could do so if we had more time.

First, we'll define the new pipeline step. Note the last line of this new function, which gives this step's pod the credentials to access GCP.

In [ ]:
# defining the new data preprocessing pipeline step. 
# Note the last line, which gives this step's pod the credentials to access GCP
def preproc_op(data_dir, project):
  return dsl.ContainerOp(
    name='datagen',
    image='gcr.io/google-samples/ml-pipeline-t2tproc',
    arguments=[ "--data-dir", data_dir, "--project", project]
  ).apply(gcp.use_gcp_secret('user-gcp-sa'))

### Modify the pipeline to add the new step

Now, we'll redefine the pipeline to add the new step.

In [ ]:
# Then define a new Pipeline. It's almost the same as the original one, 
# but with the addition of the data processing step.
import kfp.dsl as dsl
import kfp.gcp as gcp

@dsl.pipeline(
  name='Github issue summarization',
  description='TFT-based feature processing, TFMA, TFJob, CMLE OP, and TF-Serving'
)
def gh_summ2(
  train_steps: dsl.PipelineParam=dsl.PipelineParam(name='train-steps', value=2019300),
  working_dir: dsl.PipelineParam=dsl.PipelineParam(name='working-dir', value='YOUR_GCS_DIR_HERE'),
  checkpoint_dir: dsl.PipelineParam=dsl.PipelineParam(name='checkpoint-dir', value='s3://ml-demo10/ghs_data/model_output_tbase.bak2019000'),
  deploy_webapp: dsl.PipelineParam=dsl.PipelineParam(name='deploy-webapp', value='true'),
  data_dir: dsl.PipelineParam=dsl.PipelineParam(name='data-dir', value='s3://ml-demo10/ghs_data/t2t_data_gh_all/')):

  # The new pre-processing op.
  preproc = preproc_op(project=project,
      data_dir=('%s/%s/gh_data' % (working_dir, '{{workflow.name}}')))

  train = dsl.ContainerOp(
      name = 'train',
      image = 'gcr.io/google-samples/ml-pipeline-t2ttrain',
      arguments = [ "--data-dir", data_dir,
          "--checkpoint-dir", checkpoint_dir,
          "--model-dir", '%s/%s/model_output' % (working_dir, '{{workflow.name}}'),
          "--train-steps", train_steps, "--deploy-webapp" , deploy_webapp],
      file_outputs={'output': '/tmp/output'}

      ).apply(gcp.use_gcp_secret('user-gcp-sa'))
  train.after(preproc)

  serve = dsl.ContainerOp(
      name = 'serve',
      image = 'gcr.io/google-samples/ml-pipeline-kubeflow-tfserve',
      arguments = ["--model_name", 'ghsumm-%s' % ('{{workflow.name}}',),
          "--model_path", '%s/%s/model_output/export' % (working_dir, '{{workflow.name}}')
          ]
      )
  serve.after(train)
  train.set_gpu_limit(4)

  with dsl.Condition(train.output=='true'):
    webapp = dsl.ContainerOp(
        name = 'webapp',
        image = 'gcr.io/google-samples/ml-pipeline-webapp-launcher',
        arguments = ["--model_name", 'ghsumm-%s' % ('{{workflow.name}}',),
            "--github_token", github_token]

        )
    webapp.after(serve)   


### Compile the new pipeline definition and submit the run

In [ ]:
compiler.Compiler().compile(gh_summ2, 'ghsumm2.tar.gz')

In [ ]:
run = client.run_pipeline(exp.id, 'ghsumm2', 'ghsumm2.tar.gz',
                          params={'working-dir': WORKING_DIR,
                                  'github-token': GITHUB_TOKEN,
                                  'deploy-webapp': DEPLOY_WEBAPP,
                                  'project': PROJECT_NAME})

![The new pipeline.](https://storage.googleapis.com/amy-jo/images/datagen_t2t_pipeline.png)

When this new pipeline finishes running, you'll be able to see your generated processed data files in GCS under the path: `WORKING_DIR/<pipeline_name>/gh_data`. There isn't time in the workshop to pre-process the full dataset, but if there had been, we could have defined our pipeline to read from that generated directory for its training input.

-----------------------------
Copyright 2018 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.